In [5]:
import pandas as pd
import numpy as np
from pystoned import CNLS, CNLSDDF, StoNED
from pystoned.constant import CET_MULT, CET_ADDI, FUN_COST, FUN_PROD, RTS_VRS, RTS_CRS, RED_MOM, OPT_LOCAL, RED_QLE, RED_KDE

In [6]:
data = pd.read_csv("../../data_ready/data_ready.csv").replace([np.inf, -np.inf], np.nan).dropna()

In [7]:
data_pol = data[data["CNT"] == 'POL']

inputs_pol = data_pol[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs_pol = data_pol[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs_pol = data_pol[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables_pol = data_pol[["dropout_rate"]].values.tolist()
contextuals_pol = data_pol[["bol_competition", "bol_location"]].values.tolist()
data_pol["avg_scores"] = outputs_pol.mean(axis = 1)
z_output_pol = data_pol[["avg_scores"]].values.tolist()

<ipython-input-7-2d5bbaf7f3b1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pol["avg_scores"] = outputs_pol.mean(axis = 1)


In [8]:
model1_pol = CNLS.CNLS(
    y = z_output_pol, 
    x = inputs_pol, 
    z = contextuals_pol,
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_pol = CNLS.CNLS(
    y = z_output_pol, 
    x = inputs_pol, 
    z = contextuals_pol, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_pol = CNLSDDF.CNLSDDF(
    y = d_outputs_pol, 
    x = inputs_pol, 
    b = undesirables_pol, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_pol = CNLSDDF.CNLSDDF(
    y = d_outputs_pol, 
    x = inputs_pol, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

In [9]:
model1_pol.optimize(OPT_LOCAL)
model2_pol.optimize(OPT_LOCAL)
model3_pol.optimize(OPT_LOCAL)
model4_pol.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 53361           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1619            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.03    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 231
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [10]:
### inefficiency
ineff1_pol = StoNED.StoNED(model1_pol)
ineff2_pol = StoNED.StoNED(model2_pol)
ineff3_pol = StoNED.StoNED(model3_pol)
ineff4_pol = StoNED.StoNED(model4_pol)

In [11]:
res_pol = {
    "res1_pol" : {
        "alpha" : "None",
        "beta" : model1_pol.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model1_pol.get_lamda(),
        "residuals" : model1_pol.get_residual(),
        "frontier" : model1_pol.get_frontier(),
        "unconditional_expected_inefficiency" : ineff1_pol.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff1_pol.get_technical_inefficiency(method = RED_QLE)
    },


    "res2_pol" : {
        "alpha" : model2_pol.get_alpha(),
        "beta" : model2_pol.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model2_pol.get_lamda(),
        "residuals" : model2_pol.get_residual(),
        "frontier" : model2_pol.get_frontier(),
        "unconditional_expected_inefficiency" : ineff2_pol.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff2_pol.get_technical_inefficiency(method = RED_QLE)
    },


    "res3_pol" : {
        "alpha" : model3_pol.get_alpha(),
        "beta" : model3_pol.get_beta(),
        "delta" : model3_pol.get_delta(),
        "gamma" : model3_pol.get_gamma(),
        "lamda" : "None",
        "residuals" : model3_pol.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff3_pol.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff3_pol.get_technical_inefficiency(method = RED_QLE)
    },


    "res4_pol" : {
        "alpha" : "None",
        "beta" : model4_pol.get_beta(),
        "delta" : "None",
        "gamma" : model4_pol.get_gamma(),
        "lamda" : "None",
        "residuals" : model4_pol.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff4_pol.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff4_pol.get_technical_inefficiency(method = RED_QLE)
    }
}


In [12]:
import pickle
file = 'res_pol'
open_file = open(file, "wb")

pickle.dump(res_pol, open_file)
open_file.close()


# open_file = open(file, "rb")
# res_pol = pickle.load(open_file)
# open_file.close()

